# Про курс

Курс будет опираться на курс осеннего семестра "Машинное обучение в биологии", но отдельные темы повторим.

Лекции проводятся по средам, 19:00 по Москве, в удаленном формате.
Семинары тоже по средам, очно, чуть раньше.

Домашки будет две, постараемся провести их в Kaggle.

## План "Максимум"

- Block 1. Classic NLP and Retrieval
	- Text Feature Extraction. Tokenization, stemming, lemmatization, BPE, **TF-IDF**, BM25.
	- **Word2Vec**: Continuous Bag-of-Words and Skip-Gram. Revisiting transformers: Encoders vs Decoders vs Full Transformers. BERT family, GPTs, T5.
	- Vector Search. Algorithms for fast nearest neighbors search.
- Block 2. Graphs
	- Classical graph methods. Centrality, clustering, DeepWalk, Node2Vec and Matrix decomposition.
	- Graph Neural Networks. **GCN**, GAT (revisit). Heterogeneous Graphs, Knowledge Graphs, KG Decoders (TransE, ComplEx), RGCN.
	- Proteins. GNN in 3D, equivariance in deep learning. **AlphaFold 2**.
- Block 3. Reinforcement Learning
	- Intro to RL. Markov Decision Processes. Policy and Value. **Q-learning**. DQN.
	- Policy Gradient Methods. REINFORCE, Actor-Critic. **PPO**
- Block 4. Generative AI
	- LLM Applications. Sampling Methods, **RAG**, **Agents**, Chain-of-Thoughts and Co.
	- **Diffusion**. Generating images. *Schedules*.
	- Diffusion for chemical compounds, 3D structures (proteins and peptides). *Latent Diffusion Models*

# Представления слов

То, как модели машинного обучения "видят" данные, отличается от того, как это делаем мы. Например, мы можем легко понять текст "ЩУКа съела ацетат", но наши модели не могут этого сделать - им нужно какое-то представление данных, скорее всего в виде какого-то *вектора*. Такие векторы, или **эмбеддинги**, представляют собой представления слов, которые могут быть поданы в вашу модель.

Векторы полезны при работе с любыми текстами, в том числе биологическими. Векторы остатков аминокислот, нуклеиновых кислот очень часто используются.

![image](https://lena-voita.github.io/resources/lectures/word_emb/word_repr_intro-min.png)

На практике у вас есть словарь разрешенных слов, который вы выбираете заранее. Для каждого слова в словаре есть таблица поиска, содержащая его эмбеддинги.

Для учета неизвестных слов (тех, которых нет в словаре) в словаре обычно используется специальная лексема **UNK**. В качестве альтернативы неизвестные лексемы можно игнорировать или присваивать им нулевой вектор.

Главный вопрос: как мы получаем эти векторы слов?

## One-hot encoding

Самый простой способ - представить слова в виде векторов с одной точкой: для $i$-го слова в словаре вектор имеет 1 в $i$-м измерении и 0 в остальных. В машинном обучении это самый простой способ представления категориальных признаков.

![image](https://lena-voita.github.io/resources/lectures/word_emb/one_hot-min.png)

Нетрудно догадаться, почему такиe векторы - не лучший способ представления слов. Одна из проблем заключается в том, что для больших словарей эти векторы будут очень длинными: размерность вектора равна размеру словаря. На практике это нежелательно, но данная проблема не является самой важной.

На самом например, одноточечные векторы "думают", что кошка так же близка к собаке, как и стол. Можно сказать, что одноточечные векторы не улавливают смысл.

## Дистрибутивная семантика

Чтобы отразить смысл слов в их векторах, нам сначала нужно определить понятие смысла, которое можно использовать на практике.

Часто возможно извлекать смысл из **контекста**. В этом идея всех методов извлечения признаков из слов.

Самые простые из них используют то или иное матричное разложение.

### Count-based methods

![image](https://lena-voita.github.io/resources/lectures/word_emb/preneural/idea-min.png)

Общая процедура показана выше и состоит из двух шагов:

1. построить матрицу "слово-контекст"
2. уменьшить ее размерность.

Уменьшение размерности необходимо по двум причинам. Во-первых, исходная матрица очень велика. Во-вторых, поскольку многие слова встречаются лишь в нескольких возможных контекстах, матрица потенциально содержит много неинформативных элементов (по сути нулей).

Чтобы оценить сходство между словами/контекстами, обычно требуется оценить точечный произведение нормализованных векторов слов/контекстов (т.е. косинусное сходство).

Чтобы определить метод, основанный на подсчете, нам нужны две вещи:

1. возможные контексты (в том числе нужно знать, что означает появление слова в том или ином контексте)
2. понятие ассоциации, то есть формулы для вычисления элементов матрицы.



### Co-Occurence Counts

![image](https://lena-voita.github.io/resources/lectures/word_emb/preneural/window-min.png)

Самый простой подход - определить контексты как каждое слово в окне размером $L$. Элемент матрицы для пары слово-контекст ($w$, $c$) - это количество раз, когда $w$ встречается в контексте $c$. Это самый базовый (и очень, очень старый) метод получения эмбеддингов.

### PPMI

Здесь контексты определяются так же, как и раньше, но мера связи между словом и контекстом более умная: положительный PMI (или сокращенно PPMI). Мера PPMI широко рассматривается как лучшая до нейронок моделей.

![image](https://lena-voita.github.io/resources/lectures/word_emb/preneural/define_ppmi-min.png)

Было показано, что Word2Vec, с которым мы скоро познакомимся, неявно аппроксимирует факторизацию матрицы PMI.

## Latent Semantic Analysis (LSA)

**LSA** анализирует коллекцию документов, не отдельные слова. Если в предыдущих подходах контексты служили только для получения векторов слов и затем отбрасывались, то здесь нас также интересует контекст, или, в данном случае, векторы документов.

LSA - одна из простейших тематических моделей: для измерения сходства между документами можно использовать косинусоидальное сходство между векторами документов.

![image](https://lena-voita.github.io/resources/lectures/word_emb/preneural/lsa-min.png)

## Word2Vec

Word2Vec - это модель, параметрами которой являются векторы слов. Эти параметры итеративно оптимизируются для достижения определенной цели. Цель заставляет векторы слов "знать" контексты, в которых может встречаться слово: векторы обучаются предсказывать возможные контексты соответствующих слов. Как вы помните из гипотезы распределения, если векторы "знают" о контекстах, то они "знают" и значение слова.

Word2Vec - это итерационный метод. Его основная идея заключается в следующем:

- возьмите огромный корпус текстов;
- пройдите по тексту с помощью скользящего окна, перемещаясь по одному слову за раз. На каждом шаге есть центральное слово и контекстные слова (другие слова в этом окне);
- для центрального слова вычислите вероятности контекстных слов;

![image](https://lena-voita.github.io/resources/lectures/word_emb/w2v/window_prob1-min.png)

На самом деле можно делать и иначе. Существует две разновидности word2vec моделей:

* **Continuous bag-of-words model**: предсказывает среднее слово на основе окружающих контекстных слов. Контекст состоит из нескольких слов до и после текущего (среднего) слова. Эта архитектура называется моделью мешка слов, поскольку порядок слов в контексте не важен.
* **Skip-gram**: предсказывает слова в определенном диапазоне до и после текущего слова в одном и том же предложении. Ниже приведен пример работы этой модели.


Обе оптимизируют вероятнисти:

![word2vec_full_softmax](https://tensorflow.org/text/tutorials/images/word2vec_full_softmax.png)



Вычисление знаменателя этой формулы предполагает выполнение полного softmax по всему словарному запасу слов, который часто бывает большим: до десятков миллионов терминов.